# Otimização Stardew Valley

Neste notebook, o objetivo é trabalhar com algoritmos de otimização diversos, desde os clássicos até os famosos bio-inspirados. Mas antes de pensar nos algoritmos, vamos pensar no problema.

Podemos pensar em casos de otimização com muita facilidade, desde aqueles casos clássicos da literatura de roteamento entre cidades (e o famoso problema do caixeiro viajante, TSP) até os processos de decisão do nosso dia a dia, como comprar a maior diversidade de produtos no mercado com um valor fixo de dinheiro disponível. Em outras palavras, estudos de caso não faltam para a gente implementar métodos de otimização.

Dito isto, não vamos testar problemas clássicos demais aqui, como o TSP. Por outro lado, vamos otimizar as plantações de um jogo chamado Stardew Valley, um simulador de fazenda bem popular. O objetivo é, para uma determina estação e uma disponibilidade de espaços para plantio, qual a melhor configuração de cultivos maximiza o lucro. As plantas disponíveis podem ser encontradas na [wiki](https://pt.stardewvalleywiki.com/Lavouras)

In [8]:
from enum import Enum
from typing import List
from typing_extensions import Self

In [58]:
class Season(Enum):
    PRIMAVERA = 1
    VERAO = 2
    OUTONO = 3
    INVERNO = 4

class Plant:
    def __init__(self, name: str, season: Season, buy_price: int, days_yield: int,
        quantity_yield: int, sell_price: int, days_after_yield: int=None
    ) -> None:
        self.name = name
        self.season = season
        self.buy_price = buy_price
        self.days_yield = days_yield
        self.quantity_yield = quantity_yield
        self.sell_price = sell_price
        self.days_after_yield = days_after_yield

    def get_profit(self, time_period: int, log=False) -> int:
        buy = self.buy_price
        growth = 0
        sell = 0
        loop = 0

        def show(title: str, day: int) -> None:
            print(f"Ação de {title} executada no dia {day}")

        for i in range(1, time_period + 1):
            # Vende uma plantação, se ela não for perene
            if growth == self.days_yield and self.days_after_yield is None:
                sell = sell + self.quantity_yield * self.sell_price
                if log: show("venda", i)
                loop = loop + 1
                growth = 0

            # Vende uma plantação perene
            if self.days_after_yield is not None and growth == self.days_after_yield:
                sell = sell + self.quantity_yield * self.sell_price
                if log: show("venda", i)
                loop = loop + 1
                growth = 0 

            # Se não for possível mais colheitas, finaliza o loop
            if self.days_after_yield is None and (time_period - i) + growth < self.days_yield:
                break

            if self.days_after_yield is not None and (time_period - i) + growth < self.days_after_yield:
                break

            # No dia zero, depois de uma colheita, faz a replantação, se
            # a cultura não for perene
            if growth == 0 and loop > 0 and self.days_after_yield is None:
                buy = buy + self.buy_price
                if log: show("compra", i)
            
            growth = growth + 1

        return sell - buy

class Crop:
    def __init__(self, plant: Plant, quantity: int) -> None:
        self.plant = plant
        self.quantity = quantity
    
    def set_quantity(self, quantity: int) -> Self:
        self.quantity = quantity
        return self

class StardewFarm:
    def __init__(self, max_spaces: int) -> None:
        self.max_spaces = max_spaces

    def random_plants(self, season: Season) -> List[Crop]:
        pass

    def get_neighbors(self, crops: List[Crop], number: int) -> List[List[Crop]]:
        pass

In [61]:
garlic = Plant("Alho", Season.PRIMAVERA, 40, 4, 1, 60, 2)
print(garlic.get_profit(13, log=False))

320
